In [4]:
import os
import re

def parse_output_metadata_file(metadata_file_path):
    """
    output_metadata.txt 파일을 파싱하여
    { "U1": 14, "U2": 8, ... } 형태로 반환.
    (User ID -> Question 수)
    """
    user_question_count = {}
    pattern = re.compile(r'^User ID:\s+(\S+).*Question 수:\s+(\d+).*')

    with open(metadata_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            match = pattern.match(line)
            if match:
                user_id = match.group(1)   # 예: U1
                q_count = int(match.group(2))  # 예: 14
                user_question_count[user_id] = q_count

    return user_question_count

def parse_user_questions_file(file_path):
    """
    [Ux]
    Question n: a, b, c, ...
    형태의 TXT 파일을 파싱해서
    {
      "U1": {
          "Question 1": [19, 4, 9, ...],
          "Question 2": [...],
          ...
      },
      "U2": {...},
      ...
    }
    형태로 반환
    """
    user_dict = {}
    current_user = None

    user_pattern = re.compile(r'^\[(.+)\]$')  # [U1], [U2], ...
    question_pattern = re.compile(r'^Question\s+(\d+)\s*:\s*(.*)$')  # Question 1: 19,4,9,...

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            # [U1], [U2], [U3] ...
            user_match = user_pattern.match(line)
            if user_match:
                current_user = user_match.group(1)  # 예: U1
                user_dict[current_user] = {}
                continue

            # Question n: ...
            q_match = question_pattern.match(line)
            if q_match and current_user is not None:
                q_num = q_match.group(1)      # '1', '2', ...
                values_str = q_match.group(2) # '19, 4, 9, ...'

                # 콤마 구분 후 공백 제거
                splitted = [v.strip() for v in values_str.split(',')]
                # 문자열 -> 정수로 변환
                try:
                    splitted = list(map(int, splitted))
                except ValueError:
                    # 숫자로 변환 안 되는 항목이 있으면, 로깅하거나 예외 처리 가능
                    pass

                user_dict[current_user][f"Question {q_num}"] = splitted

    return user_dict

def check_negative_results(input_file, output_file, metadata_folder, purpose):
    """
    요구사항:
      1) user의 Question 수가 맞지 않음 (질문 누락)
      2) 중복된 값이 있음
      3) 1~21 범위를 벗어나는 값이 있음
      4) 각 Question 값이 10개가 아님 (원래 21개였던 것을 10개로 변경)

    단, "오류가 있는 Question만" 출력하고, 
    오류 종류와 상세(중복 값, 범위 초과 값, 개수 오류 등)는 괄호 안에 명시.
    """
    # 1. 실제 파일 경로
    input_file_path = os.path.join('../../results/gpt_result', input_file)
    output_file_path = os.path.join('../../results/error_detect', output_file)
    output_metadata_path = os.path.join('../../prompts', metadata_folder, purpose, 'metadata', 'output_metadata.txt')

    # 2. 메타데이터에서 User -> 기대 Question 수 추출
    expected_question_counts = parse_output_metadata_file(output_metadata_path)

    # 3. 결과 파일 파싱 (User별 -> 질문 별 -> 값 리스트)
    user_dict = parse_user_questions_file(input_file_path)

    # 4. 각 User 마다 오류 체크
    #    errors 리스트에 [{ 'user_id': Ux, 'error_types': [...], 'error_questions': {...} }, ... ] 형태로 저장
    errors = []

    for user_id, expected_count in expected_question_counts.items():
        question_map = user_dict.get(user_id)
        user_level_error_types = set()  # 중복, 범위 초과, 개수 오류, 질문 누락 등
        error_questions = {}  # "Question 5" -> { "answers": [...], "error_str": "...", "error_types": [...] }

        # (A) user가 결과파일에 존재 안 할 경우
        if question_map is None:
            # 질문 누락(실제 expected_count개)
            user_level_error_types.add(f'질문 누락(실제 {expected_count})개)')
            errors.append({
                'user_id': user_id,
                'error_types': list(user_level_error_types),
                'error_questions': {}
            })
            continue

        # (B) Question 수(개수) 검사
        actual_count = len(question_map)
        if actual_count < expected_count:
            user_level_error_types.add(f'질문 누락(실제 {expected_count}개)')

        # (C) 각 Question 별 오류 검사
        for q_label, answers in question_map.items():
            q_errors = []  # 해당 Question의 오류 메시지(중복/범위초과/개수오류 등)
            
            # 1) 중복 검사
            duplicates = [x for x in set(answers) if answers.count(x) > 1]
            if duplicates:
                user_level_error_types.add('중복')
                # 중복 값들을 오름차순으로 정렬해서 보여주기
                duplicates_str = ', '.join(map(str, sorted(duplicates)))
                q_errors.append(f'중복: {duplicates_str}')

            # 2) 범위 검사(1~21)
            out_of_range = [x for x in answers if x < 1 or x > 21]
            if out_of_range:
                user_level_error_types.add('범위 초과')
                oor_str = ', '.join(map(str, sorted(out_of_range)))
                q_errors.append(f'범위 초과: {oor_str}')

            # 3) 개수 검사(10개인지) -- [여기만 21 -> 10 으로 수정]
            length = len(answers)
            if length != 5:
                user_level_error_types.add('개수 오류')
                q_errors.append(f'개수 오류: {length}개')

            # q_errors가 비어있지 않다면(=해당 Question에 오류가 있다면)
            if q_errors:
                # 괄호 안에 표시할 에러 메시지들
                error_str = ', '.join(q_errors)
                error_questions[q_label] = {
                    "answers": answers,
                    "error_str": error_str
                }

        # (D) 만약 user_level_error_types나 error_questions가 비어있지 않다면 오류
        if user_level_error_types or error_questions:
            errors.append({
                'user_id': user_id,
                'error_types': sorted(list(user_level_error_types)),  # 정렬
                'error_questions': error_questions
            })

    # (E) expected_question_counts에는 없지만 결과파일에만 있는 User
    #     - 필요하다면 처리 가능. (현재 요구사항에 없으므로 생략)

    # 5. 최종 결과물 생성
    output_lines = []
    if errors:
        # 5-1) 한 줄 요약
        # ex) 오류 USER: [U72] : 범위 초과, [U637] : 범위 초과, ...
        error_user_messages = []
        for error in errors:
            user_id = error['user_id']
            # user-level에서의 주요 오류 유형만 ","로 연결
            error_type_str = ', '.join(error['error_types'])
            error_user_messages.append(f'[{user_id}] : {error_type_str}')

        # 콤마로 join
        error_user_messages_str = ', '.join(error_user_messages)
        output_lines.append(f'오류 USER: {error_user_messages_str}')

        # 5-2) User list: [72, 637, ...]
        user_numbers = [err['user_id'][1:] for err in errors]  # 'U72' -> '72'
        output_lines.append(f'USER list: [{", ".join(user_numbers)}]')

        # 5-3) 총 오류 USER: n
        output_lines.append(f'총 오류 USER: {len(errors)}\n')

        # 5-4) 각 User별 상세
        for error in errors:
            user_id = error['user_id']
            # User-level 오류를 콤마로 연결
            user_error_type_str = ', '.join(error['error_types'])
            output_lines.append(f'[{user_id}] : {user_error_type_str}')

            # error_questions: {"Question 5": {"answers": [...], "error_str": "개수 오류: 20개, 중복: 15"}, ...}
            for q_label in sorted(error['error_questions'].keys(), key=lambda x: int(x.split()[-1])):
                info = error['error_questions'][q_label]
                answers = info["answers"]
                err_msg = info["error_str"]  # 예: "개수 오류: 20개, 중복: 15"
                # 리스트를 ", "로 이어붙여 출력
                vals_str = ', '.join(str(v) for v in answers)
                # 최종 출력: Question 5 : 1, 15, 3, ... (개수 오류: 20개, 중복: 15)
                output_lines.append(f'{q_label} : {vals_str} ({err_msg})')

            output_lines.append('')  # 공백 줄

    else:
        # 오류가 전혀 없는 경우
        output_lines.append('오류가 발견되지 않았습니다.')

    # 6. 파일로 쓰기
    with open(output_file_path, 'w', encoding='utf-8') as f:
        for line in output_lines:
            f.write(line + '\n')

    print(f"검사가 완료되었습니다. 결과 파일: {output_file_path}")


In [22]:
check_negative_results(
    input_file="[250220] negative_ndcg2.txt",
    output_file="[250220] negative_ndcg2.txt",
    metadata_folder="[250217]",
    purpose="with_negative"
)


# check_negative_results(
#     input_file="[250217] positive_ndcg_no few shot2.txt",
#     output_file="[250217] positive_ndcg_no few shot2.txt",
#     metadata_folder="[250217]",
#     purpose="only_positive"
# )

검사가 완료되었습니다. 결과 파일: ../../results/error_detect\[250220] negative_ndcg2.txt
